In [1]:
import io
from IPython.nbformat import current
def execute_notebook(nbfile):
    with io.open(nbfile) as f:
        nb = current.read(f, 'json')
    ip = get_ipython()
    for cell in nb.worksheets[0].cells:
        if cell.cell_type != 'code':
            continue
        ip.run_cell(cell.input)
execute_notebook("/cellar/users/ramarty/Projects/hla_ii/bin/imports.ipynb")

/cellar/users/ramarty/anaconda/lib/python2.7/site-packages/IPython/nbformat/current.py:19: UserWarning: IPython.nbformat.current is deprecated.

- use IPython.nbformat for read/write/validate public API
- use IPython.nbformat.vX directly to composing notebooks of a particular version

  """)
/cellar/users/ramarty/anaconda/lib/python2.7/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


Populating the interactive namespace from numpy and matplotlib


In [2]:
files = [x for x in os.listdir('/cellar/users/ramarty/Data/hla_ii/presentation/affinities/oncogenes/') if 'wt' not in x]

In [25]:
len([x for x in files if 'DR' in x])

660

In [3]:
print len(files)

5620


In [4]:
f = files[0]

In [5]:
f.split('.')[0]

'HLA-DQA10404-DQB10630'

In [6]:
all_dfs = []
for i, f in enumerate(files):
    if i % 100 == 0:
        print i
    df = pd.read_csv('/cellar/users/ramarty/Data/hla_ii/presentation/affinities/tsgenes/{0}'.format(f), \
            skiprows=1, index_col=0, sep='\t')
    df['allele'] = f.split('.')[0]
    all_dfs.append(df[df.Rank < 2])
df = pd.concat(all_dfs)

0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
5000
5100
5200
5300
5400
5500
5600


In [7]:
len(df)

478526

In [8]:
df.to_csv('/cellar/users/ramarty/Data/hla_ii/generated_data/neoantigens/peptides_below_2.tsgenes.csv')

In [17]:
cutoff = 2
strength = 'strong'

In [18]:
df = pd.concat([pd.read_csv('/cellar/users/ramarty/Data/hla_ii/generated_data/neoantigens/peptides_below_{0}.csv'.format(cutoff),
                index_col=0),
                pd.read_csv('/cellar/users/ramarty/Data/hla_ii/generated_data/neoantigens/peptides_below_{0}.tsgenes.csv'.format(cutoff),
                index_col=0)])

In [19]:
def get_gene(x):
    if len(x.split('-')) > 2:
        return x.split('-')[1][:2]
    else:
        return x[:2]
df['Gene'] = df['allele'].apply(get_gene)

In [20]:
patient_mutations = pd.read_csv('/cellar/users/ramarty/Data/hla_ii/presentation/clean_matrices/patient_mutations.cancer.all.csv',
                 index_col=0)

In [21]:
mutation_counts = pd.DataFrame(patient_mutations.sum()).reset_index()
mutation_counts.columns = ['ID', 'mutation_count_in_TCGA']

In [22]:
mutation_counts.head()

,ID,mutation_count_in_TCGA
0,EGFR_ifd_55174772,13
1,NOTCH1_ifd_136518618,7
2,PIK3R1_ifd_68295304,5
3,CTNNB1_ifd_41224643,5
4,PIK3R1_ifd_68293795,4


In [23]:
df_resticted = df[df.Gene == 'DR'].groupby(['Peptide', 'ID']).count()[['allele']].sort_values('allele', ascending=False).reset_index()[:500]
df_resticted.columns = ['Peptide', 'ID', 'number_of_alleles_binding']

In [24]:
pd.merge(df_resticted, mutation_counts, on='ID', how='left').dropna() \
        .to_csv('/cellar/users/ramarty/Data/hla_ii/generated_data/neoantigens/neoantigens.{0}.csv'.format(strength))

In [63]:
well_presented_peptides = list(df.Peptide.value_counts()[df.Peptide.value_counts() > 4000].index)
len(well_presented_peptides)

38

In [64]:
df[df.Peptide.isin(well_presented_peptides)][['Peptide', 'ID']].drop_duplicates()

,Peptide,ID
Pos,,
0,DMALALESARLLTWH,ACADS_R330H
4,NEIMELLSAATFFQL,BTBD11_K1003T
5,EIMELLSAATFFQLE,BTBD11_K1003T
6,IMELLSAATFFQLEA,BTBD11_K1003T
7,MELLSAATFFQLEAL,BTBD11_K1003T
8,ELLSAATFFQLEALQ,BTBD11_K1003T
9,LLSAATFFQLEALQR,BTBD11_K1003T
10,LSAATFFQLEALQRH,BTBD11_K1003T
0,DRFRSFEALLADLTQ,DCLK1_R93Q


In [65]:
ras = 'ADGVGKSALTIQLIQ'

In [67]:
df[df.Peptide == ras].describe()

,1-log50k,nM,Rank,Ave,NB
count,4443.00000,4443.000000,4443.000000,4443.00000,4443.000000
mean,0.53807,181.945172,6.828832,0.53807,0.961512
std,0.06067,118.365332,1.699053,0.06067,0.192392
min,0.40300,36.350000,3.000000,0.40300,0.000000
25%,0.49700,91.090000,5.500000,0.49700,1.000000
50%,0.53100,160.400000,7.000000,0.53100,1.000000
75%,0.58300,232.220000,8.500000,0.58300,1.000000
max,0.66800,642.130000,9.500000,0.66800,1.000000
